# **Índice de Sentimento V1**

In [2]:
import numpy as np
import pandas as pd
import os

## *Sentimentos*

In [3]:
sent = pd.read_csv('./data/news/noticias_sentimentos.csv')
sent['Data'] = pd.to_datetime(sent['Data'], format='%Y-%m-%d')
sent.loc[sent['Sentimento'] == 'POSITIVE', 'Score Signal'] = sent.loc[sent['Sentimento'] == 'POSITIVE', 'Score']
sent.loc[sent['Sentimento'] == 'NEGATIVE', 'Score Signal'] = sent.loc[sent['Sentimento'] == 'NEGATIVE', 'Score'] * -1
sent.loc[sent['Sentimento'] == 'NEUTRAL', 'Score Signal'] = 0
sent

,Data,Notícia,Sentimento,Score,Score Signal
0,2024-09-03,Unipar e MELVER estão com inscrições abertas p...,NEUTRAL,0.700441,0.000000
1,2024-09-03,Mercado financeiro hoje: Vale (VALE3) em busca...,NEGATIVE,0.515265,-0.515265
2,2024-09-03,Briga entre X e STF pode afastar investidores ...,NEGATIVE,0.850306,-0.850306
3,2024-09-03,Podcast | Fechamento de Mercado - 03/09/2024,NEUTRAL,0.614209,0.000000
4,2024-09-03,Comentario semanal - Perspectivas,NEUTRAL,0.610426,0.000000
...,...,...,...,...,...
84846,2024-09-03,Bolsa recua com peso da Vale e dados nos EUA; ...,NEGATIVE,0.830644,-0.830644
84847,2024-09-03,PIB do Brasil cresce acima do esperado no 2º t...,POSITIVE,0.772979,0.772979
84848,2024-09-03,Investidores estrangeiros ingressam com R$ 10 ...,POSITIVE,0.731023,0.731023
84849,2024-09-03,Expert XP: como é ser um investidor de curto p...,NEUTRAL,0.667090,0.000000


In [ ]:
# ### Código antigo Guigas ###

# sentimentos_g = []
# sentimentos_t = []
# for data in sent['Data'].unique():
#     dia = sent[sent['Data'] == data]
#     if dia['Sentimento'].value_counts().shape[0] != 1:
#         total_dia_g = dia[dia['Sentimento'] != 'NEUTRAL']['Score'].sum() + dia[dia['Sentimento'] == 'NEUTRAL']['Score'].sum() * 0.5
#         total_dia_t = len(dia[dia['Sentimento'] != 'NEUTRAL']) + len(dia[dia['Sentimento'] == 'NEUTRAL']) * 0.5
#         soma_dia = dia['Score Signal'].sum()
#         sentimentos_g.append(soma_dia/total_dia_g)
#         sentimentos_t.append(soma_dia/total_dia_t)
#     else:
#         media = dia['Score Signal'].mean()
#         sentimentos_g.append(media)         
#         sentimentos_t.append(media)
        
# sentimentos_df = pd.DataFrame(sentimentos_g, index=sent['Data'].unique())
# sentimentos_df.columns = ['Sentimento Guigas']
# sentimentos_df['Sentimento Theozão'] = sentimentos_t
# sentimentos_df.describe()

In [ ]:
def indicador_sentimento(freq='D'):
    def agg_score(group):
        if group['Sentimento'].nunique() > 1:
            signal = group['Score Signal'].sum() / group['Score Modificado'].sum()
            return signal
        else:
            return group['Score Signal'].mean()

    sent['Score Modificado'] = np.where(sent['Sentimento'] == "NEUTRAL", 0.5, 1) * sent['Score']
    sentimentos_df = sent.resample(freq, on='Data').apply(agg_score).to_frame(name='Signal')
    return sentimentos_df

In [6]:
import matplotlib.pyplot as plt
import yfinance as yf

ticker = 'BOVA11.SA'

dados = yf.download(ticker, start=min(sentimentos_df.index), end=max(sentimentos_df.index))['Adj Close']

fig, ax = plt.subplots(figsize=(15, 5))

ax2 = ax.twinx()
ax.plot(dados, color='black')
ax2.plot(sentimentos_df['Signal'], color='blue')

plt.show()

NameError: name 'sentimentos_df' is not defined

In [7]:
dados_agg = dados.resample('W').mean()
df_dados = dados_agg.to_frame().merge(sentimentos_df, left_on='Date', right_on='Data', how='left')
df_dados['Signal'].corr(df_dados['Adj Close'])

0.5745814689809325

In [10]:
sentimentos_df.to_csv('./data/news/sentimentos.csv')

## *Índice de Sentimento*

EmotiWand = 0.25(Volatilidade) + 0.25(Momentum e Volume) + 0.40(Sentimentos) + 0.10(Trends)

In [196]:
import pandas as pd
import numpy as np

vix = pd.read_excel('./data/vix/PerformanceGraphExport.xls')
vol = pd.read_excel('./data/vix/IVol-BR.xls')
vol_mom = pd.read_csv('./data/momentum_vol/ibov_vol_rsi.csv')
sent = pd.read_csv('./data/news/noticias_sentimentos.csv')
trends = pd.read_csv('./data/trends/interesses_google.csv')

In [197]:
import datetime as dt

vix['Data'] = pd.to_datetime(vix['Effective date '])
vol['Data'] = vol[['year', 'month', 'day']].apply(lambda x: dt.datetime(x.iloc[0], x.iloc[1], x.iloc[2]), axis=1)
vol_mom['Data'] = pd.to_datetime(vol_mom['Date'])
trends['Data'] = pd.to_datetime(trends['date'])
sent['Data'] = pd.to_datetime(sent['Data'])

start = sent['Data'].min().strftime('%Y-%m-%d')
end = sent['Data'].max().strftime('%Y-%m-%d')
print(f'{start} - {end}')

2014-09-08 - 2024-09-08


In [198]:
sent.loc[sent['Sentimento'] == 'POSITIVE', 'Score Signal'] = sent.loc[sent['Sentimento'] == 'POSITIVE', 'Score']
sent.loc[sent['Sentimento'] == 'NEGATIVE', 'Score Signal'] = sent.loc[sent['Sentimento'] == 'NEGATIVE', 'Score'] * -1
sent.loc[sent['Sentimento'] == 'NEUTRAL', 'Score Signal'] = 0

def indicador_sentimento(df, freq='D'):
    def agg_score(group):
        if group['Sentimento'].nunique() > 1:
            signal = group['Score Signal'].sum() / group['Score Modificado'].sum()
            return signal
        else:
            return group['Score Signal'].mean()

    df['Score Modificado'] = np.where(df['Sentimento'] == "NEUTRAL", 0.5, 1) * df['Score']
    sentimentos_df = df.resample(freq, on='Data').apply(agg_score).to_frame(name='Signal')
    return sentimentos_df

sent = indicador_sentimento(sent)

In [199]:
vix['ivolbr'] = vix['S&P/B3 Ibovespa VIX']
vix_total = pd.concat([vix, vol])[['Data', 'ivolbr']].sort_values('Data')

In [200]:
df = pd.merge_ordered(vix_total, vol_mom, on='Data', how='outer')
df = pd.merge_ordered(df, sent, on='Data', how='outer')
df = pd.merge_ordered(df, trends, on='Data', how='outer')

df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

C:\Users\Andre\AppData\Local\Temp\ipykernel_12576\1051337680.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\Andre\AppData\Local\Temp\ipykernel_12576\1051337680.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [201]:
df.drop(columns=['Date', 'date'], inplace=True)
df.set_index('Data', inplace=True)
df['trends'] = df.drop(columns=['ivolbr', 'RSI_14', 'Volume', 'Signal']).mean(axis=1)
df = df[['ivolbr', 'RSI_14', 'Volume', 'Signal', 'trends']]
df = df[(df.index >= start) & (df.index <= end)]
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3805 entries, 2014-09-08 to 2024-09-08
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ivolbr  3805 non-null   float64
 1   RSI_14  3805 non-null   float64
 2   Volume  3805 non-null   float64
 3   Signal  3805 non-null   float64
 4   trends  3805 non-null   float64
dtypes: float64(5)
memory usage: 178.4 KB


In [202]:
df = df.resample('D').mean().sort_values('Data')

In [206]:
import pandas as pd

df = pd.read_csv('./data/indicadores/indicadores.csv')

cols_to_scale = ['ivolbr', 'Volume', 'Signal']
cols_not_to_scale = df.columns[df.columns != 'Data'][~df.columns[df.columns != 'Data'].isin(cols_to_scale)]

scaled_data = []

for idx in range(step-1, len(df)):
    grupo = df.iloc[idx - step + 1:idx + 1, df.columns.get_indexer(cols_to_scale)]
    scaled_grupo = scaler.fit_transform(grupo)
    non_scaled_values = df.iloc[idx][cols_not_to_scale].values
    scaled_data.append(list(scaled_grupo[-1]) + list(non_scaled_values))

scaled_df = pd.DataFrame(scaled_data, columns=list(cols_to_scale) + list(cols_not_to_scale), index=df.iloc[step-1:].index)

scaled_df['Data'] = df.iloc[step-1:]['Data'].values

scaled_df.head()

,ivolbr,Volume,Signal,RSI_14,trends,Data
251,0.086642,0.169503,0.569611,62.494222,2.810345,2015-05-17
252,0.043212,0.313317,0.464255,54.222463,2.810345,2015-05-18
253,0.074838,0.222962,0.580549,49.465619,2.810345,2015-05-19
254,0.160838,0.232023,0.134349,45.795663,2.810345,2015-05-20
255,0.169499,0.236090,0.333761,47.281883,2.810345,2015-05-21


In [207]:
scaled_df[cols_to_scale] = scaled_df[cols_to_scale] * 100

# Alterar lado
# scaled_df[['ivolbr', 'RSI_14']] = 100 - scaled_df[['ivolbr', 'RSI_14']]

In [210]:
scaled_df.head()

,ivolbr,Volume,Signal,RSI_14,trends,Data
251,8.664162,16.950283,56.961117,62.494222,2.810345,2015-05-17
252,4.321185,31.331750,46.425522,54.222463,2.810345,2015-05-18
253,7.483823,22.296154,58.054924,49.465619,2.810345,2015-05-19
254,16.083801,23.202317,13.434859,45.795663,2.810345,2015-05-20
255,16.949932,23.609032,33.376132,47.281883,2.810345,2015-05-21


In [211]:
scaled_df.to_csv('./data/indicadores/indicadores_scaled.csv', index=False)